In [1]:
import datetime
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data.csv')

df_kaggle = pd.read_csv('data_kaggle.csv')
df_kaggle.rename(columns={'artist_name':'artist', 'track_name':'name', 'track_id':'id'}, inplace=True)
df_kaggle.drop_duplicates(inplace=True)

print(df.shape)
print(df_kaggle.shape)

(62179, 19)
(232725, 19)


In [3]:
df_ids = df['id'].values.tolist()
df_kaggle_ids = df_kaggle['id'].values.tolist()
ids_to_drop = set(df_ids).intersection(set(df_kaggle_ids))

df.drop(df[df.id.isin(ids_to_drop)].index, inplace=True)

df = df.append(df_kaggle)
df.reset_index(drop=True, inplace=True)

print(len(ids_to_drop))
print(df.shape)

19480
(275424, 19)


In [4]:
df['duration_min'] = df['duration_ms']/60000
df['duration_min'] = df['duration_min'].round(2)

In [5]:
def validate(row):
    try:
        datetime.datetime.strptime(row.release_date, '%Y-%m-%d')
        return True
    except ValueError:
        return False

df = df[df.apply(lambda x: validate(x), axis = 1)]
df['year'] = pd.DatetimeIndex(df['release_date']).year

In [6]:
df['decade'] = None
df.loc[df['year'] < 1960, 'decade'] = '<1960s'
df.loc[(df['year'] >= 1960) & (df['year'] < 1970), 'decade'] = '1960s'
df.loc[(df['year'] >= 1970) & (df['year'] < 1980), 'decade'] = '1970s'
df.loc[(df['year'] >= 1980) & (df['year'] < 1990), 'decade'] = '1980s'
df.loc[(df['year'] >= 1990) & (df['year'] < 2000), 'decade'] = '1990s'
df.loc[(df['year'] >= 2000) & (df['year'] < 2010), 'decade'] = '2000s'
df.loc[(df['year'] >= 2010) & (df['year'] < 2020), 'decade'] = '2010s'
df.loc[(df['year'] >= 2020) & (df['year'] < 2030), 'decade'] = '2020s'
df.decade.value_counts()

2010s     175243
2000s      46246
1990s      17157
1980s       5067
1970s       4651
1960s       2211
<1960s       637
2020s          3
Name: decade, dtype: int64

In [7]:
df = df[df['year'] >= 1960]
df = df[df['year'] < 2020]
df = df.drop(labels=['duration_ms', 'release_date', 'decade', 'time_signature'], axis=1)

In [8]:
df['key'].replace(to_replace=['A', 'A#', 'B', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#'], value=[i for i in range(12)], inplace=True)
df['mode'].replace(to_replace=['Minor', 'Major'], value=[0, 1], inplace=True)

In [9]:
len(df)

250575

In [10]:
df.to_csv('clean_data.csv', sep=',', index=False)